# <a name="C2">Import des librairies


In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import seaborn as sns
import datetime as dt
import scipy.stats as st
import re
import time
from datetime import date
from datetime import datetime
pd.options.display.float_format = "{:.2f}".format
warnings.simplefilter("ignore")

# <a name="C2">Import des fichiers et nettoyage des données

## <span style='color:green'> I - Fichier Disponibilité_Alimentaire  </span>

In [2]:
#Import du fichier
Disponibilite_Alimentaire=pd.read_csv('DisponibiliteAlimentaire_2017.csv')
Disponibilite_Alimentaire.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.00,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.00,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.00,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.00,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.00,S,Données standardisées


In [3]:
# Vérification des informations du data frame
Disponibilite_Alimentaire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176600 entries, 0 to 176599
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            176600 non-null  object 
 1   Domaine                 176600 non-null  object 
 2   Code zone               176600 non-null  int64  
 3   Zone                    176600 non-null  object 
 4   Code Élément            176600 non-null  int64  
 5   Élément                 176600 non-null  object 
 6   Code Produit            176600 non-null  int64  
 7   Produit                 176600 non-null  object 
 8   Code année              176600 non-null  int64  
 9   Année                   176600 non-null  int64  
 10  Unité                   176600 non-null  object 
 11  Valeur                  176600 non-null  float64
 12  Symbole                 176600 non-null  object 
 13  Description du Symbole  176600 non-null  object 
dtypes: float64(1), int64

In [4]:
# Vérification des statistiques des colonnes
Disponibilite_Alimentaire.describe()

,Code zone,Code Élément,Code Produit,Code année,Année,Valeur
count,176600.00,176600.00,176600.00,176600.00,176600.00,176600.00
mean,125.47,3810.05,2628.78,2017.00,2017.00,210.55
std,72.94,2211.79,91.99,0.00,0.00,4762.05
min,1.00,645.00,2511.00,2017.00,2017.00,-10388.00
25%,63.00,684.00,2556.00,2017.00,2017.00,0.00
50%,120.00,5142.00,2612.00,2017.00,2017.00,0.12
75%,188.00,5301.00,2732.00,2017.00,2017.00,8.46
max,276.00,5911.00,2899.00,2017.00,2017.00,758548.00


In [5]:
# Sélection des colonnes à conserver
Disponibilite_Alimentaire=Disponibilite_Alimentaire[['Zone','Élément','Produit','Valeur']]
Disponibilite_Alimentaire.head()

,Zone,Élément,Produit,Valeur
0,Afghanistan,Production,Blé et produits,4281.00
1,Afghanistan,Importations - Quantité,Blé et produits,2302.00
2,Afghanistan,Variation de stock,Blé et produits,-119.00
3,Afghanistan,Exportations - Quantité,Blé et produits,0.00
4,Afghanistan,Disponibilité intérieure,Blé et produits,6701.00


In [6]:
# Affichage des valeurs uniques de la colonne Produit
Disponibilite_Alimentaire['Produit'].unique()

array(['Blé et produits', 'Riz et produits', 'Orge et produits',
       'Maïs et produits', 'Seigle et produits', 'Avoine',
       'Millet et produits', 'Sorgho et produits', 'Céréales, Autres',
       'Pommes de Terre et produits', 'Ignames', 'Racines nda',
       'Sucre, canne', 'Sucre, betterave', 'Sucre Eq Brut',
       'Edulcorants Autres', 'Miel', 'Haricots', 'Pois',
       'Légumineuses Autres et produits', 'Noix et produits', 'Soja',
       'Arachides Decortiquees', 'Graines de tournesol',
       'Graines Colza/Moutarde', 'Graines de coton', 'Coco (Incl Coprah)',
       'Sésame', 'Olives', 'Plantes Oleiferes, Autre', 'Huile de Soja',
       "Huile d'Arachide", 'Huile de Tournesol',
       'Huile de Colza&Moutarde', 'Huile Graines de Coton',
       'Huile de Palmistes', 'Huile de Palme', 'Huile de Coco',
       'Huile de Sésame', "Huile d'Olive", 'Huile de Son de Riz',
       'Huile de Germe de Maïs', 'Huil Plantes Oleif Autr',
       'Tomates et produits', 'Oignons', 'Légumes, 

In [7]:
# Filtre sur la viande volaille
Disponibilite_Alimentaire_volaille=Disponibilite_Alimentaire.loc[Disponibilite_Alimentaire['Produit']=='Viande de Volailles']
Disponibilite_Alimentaire_volaille.head()

,Zone,Élément,Produit,Valeur
651,Afghanistan,Production,Viande de Volailles,28.00
652,Afghanistan,Importations - Quantité,Viande de Volailles,29.00
653,Afghanistan,Variation de stock,Viande de Volailles,0.00
654,Afghanistan,Disponibilité intérieure,Viande de Volailles,57.00
655,Afghanistan,Pertes,Viande de Volailles,2.00


In [8]:
# Pivot de la colonne Élément 
Disponibilite_Alimentaire_volaille=Disponibilite_Alimentaire_volaille.pivot_table(index='Zone', columns='Élément', values = 'Valeur').reset_index()
Disponibilite_Alimentaire_volaille.head()

Élément,Zone,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,NaN,NaN,NaN,5.00,1.53,0.33,0.54,57.00,NaN,29.00,55.00,2.00,28.00,0.00,NaN,NaN,0.00
1,Afrique du Sud,0.00,NaN,NaN,143.00,35.69,9.25,14.11,2118.00,63.00,514.00,2035.00,83.00,1667.00,0.00,NaN,NaN,0.00
2,Albanie,NaN,NaN,NaN,85.00,16.36,6.45,6.26,47.00,0.00,38.00,47.00,NaN,13.00,0.00,NaN,NaN,4.00
3,Algérie,0.00,NaN,NaN,22.00,6.38,1.50,1.97,277.00,0.00,2.00,264.00,13.00,275.00,0.00,NaN,NaN,0.00
4,Allemagne,NaN,NaN,NaN,71.00,19.47,4.16,7.96,1739.00,646.00,842.00,1609.00,NaN,1514.00,-38.00,NaN,167.00,-29.00


In [9]:
# Détection des valeurs manquantes
print(Disponibilite_Alimentaire_volaille.isnull().sum())

Élément
Zone                                                               0
Alimentation pour touristes                                       94
Aliments pour animaux                                            171
Autres utilisations (non alimentaire)                            138
Disponibilité alimentaire (Kcal/personne/jour)                     0
Disponibilité alimentaire en quantité (kg/personne/an)             0
Disponibilité de matière grasse en quantité (g/personne/jour)      0
Disponibilité de protéines en quantité (g/personne/jour)           0
Disponibilité intérieure                                           2
Exportations - Quantité                                           37
Importations - Quantité                                            2
Nourriture                                                         2
Pertes                                                           105
Production                                                         4
Résidus                   

In [10]:
#Séléction des colonnes qui nous intéressent et remplacement des valeurs nulles par les moyennes de leurs colonnes
Disponibilite_Alimentaire_volaille=Disponibilite_Alimentaire_volaille[['Zone',"Disponibilité alimentaire en quantité (kg/personne/an)",'Disponibilité intérieure','Exportations - Quantité','Importations - Quantité','Nourriture','Production','Variation de stock']].fillna(Disponibilite_Alimentaire_volaille.mean())
Disponibilite_Alimentaire_volaille.head()

Élément,Zone,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Production,Variation de stock
0,Afghanistan,1.53,57.00,132.19,29.00,55.00,28.00,0.00
1,Afrique du Sud,35.69,2118.00,63.00,514.00,2035.00,1667.00,0.00
2,Albanie,16.36,47.00,0.00,38.00,47.00,13.00,4.00
3,Algérie,6.38,277.00,0.00,2.00,264.00,275.00,0.00
4,Allemagne,19.47,1739.00,646.00,842.00,1609.00,1514.00,-29.00


In [11]:
# Vérification du data frame modifié
Disponibilite_Alimentaire_volaille.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Zone                                                    172 non-null    object 
 1   Disponibilité alimentaire en quantité (kg/personne/an)  172 non-null    float64
 2   Disponibilité intérieure                                172 non-null    float64
 3   Exportations - Quantité                                 172 non-null    float64
 4   Importations - Quantité                                 172 non-null    float64
 5   Nourriture                                              172 non-null    float64
 6   Production                                              172 non-null    float64
 7   Variation de stock                                      172 non-null    float64
dtypes: float64(7), object(1)
memory usage: 10

In [12]:
# Détection des doublons
Disponibilite_Alimentaire_volaille.loc[Disponibilite_Alimentaire_volaille['Zone'].duplicated(keep=False),:]

Élément,Zone,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Production,Variation de stock


Il n'y a pas de doublons.

In [13]:
# Renommer les colonnes du fichier et supprimer colonne variation de stock
Disponibilite_Alimentaire_volaille = Disponibilite_Alimentaire_volaille.rename(columns={"Disponibilité alimentaire en quantité (kg/personne/an)":'Disponibilité alimentaire','Zone':'Pays',"Exportations - Quantité":'Exportations',"Importations - Quantité":"Importations", "Nourriture" : "Consommation"})
Disponibilite_Alimentaire_volaille.drop(columns=['Variation de stock','Disponibilité intérieure'],inplace=True)
Disponibilite_Alimentaire_volaille

Élément,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production
0,Afghanistan,1.53,132.19,29.00,55.00,28.00
1,Afrique du Sud,35.69,63.00,514.00,2035.00,1667.00
2,Albanie,16.36,0.00,38.00,47.00,13.00
3,Algérie,6.38,0.00,2.00,264.00,275.00
4,Allemagne,19.47,646.00,842.00,1609.00,1514.00
...,...,...,...,...,...,...
167,Émirats arabes unis,43.47,94.00,433.00,412.00,48.00
168,Équateur,19.31,0.00,0.00,324.00,340.00
169,États-Unis d'Amérique,55.68,3692.00,123.00,18100.00,21914.00
170,Éthiopie,0.13,132.19,1.00,14.00,14.00


In [14]:
# Création d'une liste des pays pour comparer avec autres fichiers
Liste_pays_dispo=Disponibilite_Alimentaire_volaille['Pays'].unique()
Liste_pays_dispo

array(['Afghanistan', 'Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne',
       'Angola', 'Antigua-et-Barbuda', 'Arabie saoudite', 'Argentine',
       'Arménie', 'Australie', 'Autriche', 'Azerbaïdjan', 'Bahamas',
       'Bangladesh', 'Barbade', 'Belgique', 'Belize',
       'Bolivie (État plurinational de)', 'Bosnie-Herzégovine',
       'Botswana', 'Brésil', 'Bulgarie', 'Burkina Faso', 'Bélarus',
       'Bénin', 'Cabo Verde', 'Cambodge', 'Cameroun', 'Canada', 'Chili',
       'Chine - RAS de Hong-Kong', 'Chine - RAS de Macao',
       'Chine, Taiwan Province de', 'Chine, continentale', 'Chypre',
       'Colombie', 'Congo', 'Costa Rica', 'Croatie', 'Cuba',
       "Côte d'Ivoire", 'Danemark', 'Djibouti', 'Dominique',
       'El Salvador', 'Espagne', 'Estonie', 'Eswatini', 'Fidji',
       'Finlande', 'France', 'Fédération de Russie', 'Gabon', 'Gambie',
       'Ghana', 'Grenade', 'Grèce', 'Guatemala', 'Guinée',
       'Guinée-Bissau', 'Guyana', 'Géorgie', 'Haïti', 'Honduras',
       'Hongri

## <span style='color:green'> II - Fichier Population  </span>

In [15]:
#Import du fichier
Population=pd.read_csv('Population_2000_2018.csv')
Population.head(200)


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.95,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.99,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.77,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.87,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.68,X,Sources internationales sûres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,OA,Séries temporelles annuelles,194,Arabie saoudite,511,Population totale,3010,Population-Estimations,2005,2005,1000 personnes,23816.18,X,Sources internationales sûres,NaN
196,OA,Séries temporelles annuelles,194,Arabie saoudite,511,Population totale,3010,Population-Estimations,2006,2006,1000 personnes,24498.31,X,Sources internationales sûres,NaN
197,OA,Séries temporelles annuelles,194,Arabie saoudite,511,Population totale,3010,Population-Estimations,2007,2007,1000 personnes,25184.60,X,Sources internationales sûres,NaN
198,OA,Séries temporelles annuelles,194,Arabie saoudite,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,25888.54,X,Sources internationales sûres,NaN


In [16]:
# Vérification des informations du data frame
Population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            4411 non-null   object 
 1   Domaine                 4411 non-null   object 
 2   Code zone               4411 non-null   int64  
 3   Zone                    4411 non-null   object 
 4   Code Élément            4411 non-null   int64  
 5   Élément                 4411 non-null   object 
 6   Code Produit            4411 non-null   int64  
 7   Produit                 4411 non-null   object 
 8   Code année              4411 non-null   int64  
 9   Année                   4411 non-null   int64  
 10  Unité                   4411 non-null   object 
 11  Valeur                  4411 non-null   float64
 12  Symbole                 4411 non-null   object 
 13  Description du Symbole  4411 non-null   object 
 14  Note                    258 non-null    

In [17]:
# Vérification des statistiques des colonnes
Population.describe()

,Code zone,Code Élément,Code Produit,Code année,Année,Valeur
count,4411.00,4411.00,4411.00,4411.00,4411.00,4411.00
mean,132.20,511.00,3010.00,2009.07,2009.07,29630.04
std,75.85,0.00,0.00,5.48,5.48,123802.86
min,1.00,511.00,3010.00,2000.00,2000.00,0.79
25%,68.00,511.00,3010.00,2004.00,2004.00,392.19
50%,132.00,511.00,3010.00,2009.00,2009.00,4764.74
75%,195.00,511.00,3010.00,2014.00,2014.00,18215.48
max,299.00,511.00,3010.00,2018.00,2018.00,1427647.79


In [18]:
# Sélection des colonnes à conserver
Population=Population[['Zone','Année','Valeur']]
Population.head()

,Zone,Année,Valeur
0,Afghanistan,2000,20779.95
1,Afghanistan,2001,21606.99
2,Afghanistan,2002,22600.77
3,Afghanistan,2003,23680.87
4,Afghanistan,2004,24726.68


In [19]:
# Calcul population totale car valeur pour 1000 personnes
Population['Valeur']=Population['Valeur']*1000
Population.head()

,Zone,Année,Valeur
0,Afghanistan,2000,20779953.00
1,Afghanistan,2001,21606988.00
2,Afghanistan,2002,22600770.00
3,Afghanistan,2003,23680871.00
4,Afghanistan,2004,24726684.00


In [20]:
# Conversion type colonne categ en integer
Population['Valeur'] = Population['Valeur'].astype('int')
Population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Zone    4411 non-null   object
 1   Année   4411 non-null   int64 
 2   Valeur  4411 non-null   int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 86.3+ KB


In [21]:
# Renommer les colonnes du fichier 
Population= Population.rename(columns={'Zone':'Pays',"Valeur":"Nombre d'habitants"})
Population

,Pays,Année,Nombre d'habitants
0,Afghanistan,2000,20779953
1,Afghanistan,2001,21606988
2,Afghanistan,2002,22600770
3,Afghanistan,2003,23680871
4,Afghanistan,2004,24726684
...,...,...,...
4406,Zimbabwe,2014,13586707
4407,Zimbabwe,2015,13814629
4408,Zimbabwe,2016,14030331
4409,Zimbabwe,2017,14236595


In [22]:
# Détection des valeurs manquantes
print(Population.isnull().sum())

Pays                  0
Année                 0
Nombre d'habitants    0
dtype: int64


Pas de valeurs manquantes

In [23]:
# Détection des doublons
Population.loc[Population[['Pays','Année']].duplicated(keep=False),:]

,Pays,Année,Nombre d'habitants


Il n'y a pas de doublons

In [24]:
# Création d'une liste des pays pour comparer avec autres fichiers
Liste_pays_population=Population['Pays'].unique()
Liste_pays_population

array(['Afghanistan', 'Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne',
       'Andorre', 'Angola', 'Anguilla', 'Antigua-et-Barbuda',
       'Antilles néerlandaises (ex)', 'Arabie saoudite', 'Argentine',
       'Arménie', 'Aruba', 'Australie', 'Autriche', 'Azerbaïdjan',
       'Bahamas', 'Bahreïn', 'Bangladesh', 'Barbade', 'Bélarus',
       'Belgique', 'Belize', 'Bénin', 'Bermudes', 'Bhoutan',
       'Bolivie (État plurinational de)',
       'Bonaire, Saint-Eustache et Saba', 'Bosnie-Herzégovine',
       'Botswana', 'Brésil', 'Brunéi Darussalam', 'Bulgarie',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodge', 'Cameroun',
       'Canada', 'Chili', 'Chine - RAS de Hong-Kong',
       'Chine - RAS de Macao', 'Chine, continentale',
       'Chine, Taiwan Province de', 'Chypre', 'Colombie', 'Comores',
       'Congo', 'Costa Rica', "Côte d'Ivoire", 'Croatie', 'Cuba',
       'Curaçao', 'Danemark', 'Djibouti', 'Dominique', 'Égypte',
       'El Salvador', 'Émirats arabes unis', 'Équate

In [25]:
# Renommer Soudan (ex) par Soudan
Population['Pays']= Population['Pays'].replace(["Soudan (ex)"],['Soudan'])


In [26]:
# Comparaison des 2 listes de pays
Différences_pays_dispo_population = Population.loc[~Population['Pays'].isin(Liste_pays_dispo)]
Différences_pays_dispo_population['Pays'].unique()

array(['Andorre', 'Anguilla', 'Antilles néerlandaises (ex)', 'Aruba',
       'Bahreïn', 'Bermudes', 'Bhoutan',
       'Bonaire, Saint-Eustache et Saba', 'Brunéi Darussalam', 'Burundi',
       'Comores', 'Curaçao', 'Érythrée', 'Gibraltar', 'Groenland',
       'Guadeloupe', 'Guam', 'Guinée équatoriale', 'Guyane française',
       'Île de Man', 'Îles Anglo-Normandes', 'Îles Caïmanes', 'Îles Cook',
       'Îles Falkland (Malvinas)', 'Îles Féroé', 'Îles Mariannes du Nord',
       'Îles Marshall', 'Îles Turques-et-Caïques',
       'Îles Vierges américaines', 'Îles Vierges britanniques',
       'Îles Wallis-et-Futuna', 'Libye', 'Liechtenstein', 'Martinique',
       'Mayotte', 'Micronésie (États fédérés de)', 'Monaco', 'Montserrat',
       'Nauru', 'Nioué', 'Palaos', 'Palestine',
       'Papouasie-Nouvelle-Guinée', 'Porto Rico', 'Qatar',
       'République arabe syrienne', 'République démocratique du Congo',
       'Réunion', 'Sahara occidental', 'Saint-Barthélemy',
       'Sainte-Hélène, Asce

In [27]:
# Nombre de pays différents
Différences_pays_dispo_population['Pays'].nunique()

65

Il y a plus de pays dans le fichier population.

In [28]:
# Calcul de l'évolution de la population
Population=Population.pivot_table(index = 'Pays', columns = 'Année', values = "Nombre d'habitants").reset_index()
Population["Evolution de la population (en %) entre 2000 et 2018"]=(Population[2018]-Population[2000])/(Population[2000])*100
Population.head()


Année,Pays,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Evolution de la population (en %) entre 2000 et 2018
0,Afghanistan,20779953.00,21606988.00,22600770.00,23680871.00,24726684.00,25654277.00,26433049.00,27100536.00,27722276.00,...,29185507.00,30117413.00,31161376.00,32269589.00,33370794.00,34413603.00,35383032.00,36296113.00,37171921.00,78.88
1,Afrique du Sud,44967708.00,45571274.00,46150913.00,46719196.00,47291610.00,47880601.00,48489459.00,49119759.00,49779471.00,...,51216964.00,52003755.00,52832658.00,53687121.00,54544186.00,55386367.00,56207646.00,57009756.00,57792518.00,28.52
2,Albanie,3129243.00,3129704.00,3126187.00,3118023.00,3104892.00,3086810.00,3063021.00,3033998.00,3002678.00,...,2948023.00,2928592.00,2914096.00,2903790.00,2896305.00,2890513.00,2886438.00,2884169.00,2882740.00,-7.88
3,Algérie,31042235.00,31451514.00,31855109.00,32264157.00,32692163.00,33149724.00,33641002.00,34166972.00,34730608.00,...,35977455.00,36661445.00,37383895.00,38140133.00,38923692.00,39728025.00,40551392.00,41389189.00,42228408.00,36.04
4,Allemagne,81400882.00,81453885.00,81535122.00,81614380.00,81646474.00,81602741.00,81472226.00,81277830.00,81065752.00,...,80827002.00,80855632.00,80972628.00,81174367.00,81450378.00,81787411.00,82193768.00,82658409.00,83124418.00,2.12


In [29]:
# Sélection des colonnes du dataframe
Population=Population[['Pays',2017,"Evolution de la population (en %) entre 2000 et 2018"]]
Population= Population.rename(columns={2017:"Nombre d'habitants"})
Population

Année,Pays,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018
0,Afghanistan,36296113.00,78.88
1,Afrique du Sud,57009756.00,28.52
2,Albanie,2884169.00,-7.88
3,Algérie,41389189.00,36.04
4,Allemagne,82658409.00,2.12
...,...,...,...
232,Îles Salomon,636039.00,58.21
233,Îles Turques-et-Caïques,37115.00,86.79
234,Îles Vierges américaines,104751.00,-3.72
235,Îles Vierges britanniques,29577.00,46.73


### _- Fusion des fichiers Disponibilite_Alimentaire_volaille & Population_

In [30]:
# Jointure à gauche, 'left' utilise toutes les combinaisons de clés figurant dans le tableau de gauche
df_dispo_population=pd.merge(Disponibilite_Alimentaire_volaille,Population,on=['Pays'],how='left')
df_dispo_population.head()

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018
0,Afghanistan,1.53,132.19,29.00,55.00,28.00,36296113.00,78.88
1,Afrique du Sud,35.69,63.00,514.00,2035.00,1667.00,57009756.00,28.52
2,Albanie,16.36,0.00,38.00,47.00,13.00,2884169.00,-7.88
3,Algérie,6.38,0.00,2.00,264.00,275.00,41389189.00,36.04
4,Allemagne,19.47,646.00,842.00,1609.00,1514.00,82658409.00,2.12


In [31]:
df_dispo_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Pays                                                  172 non-null    object 
 1   Disponibilité alimentaire                             172 non-null    float64
 2   Exportations                                          172 non-null    float64
 3   Importations                                          172 non-null    float64
 4   Consommation                                          172 non-null    float64
 5   Production                                            172 non-null    float64
 6   Nombre d'habitants                                    172 non-null    float64
 7   Evolution de la population (en %) entre 2000 et 2018  170 non-null    float64
dtypes: float64(7), object(1)
memory usage: 12.1+ KB


In [32]:
# Détection des valeurs manquantes
print(df_dispo_population.isnull().sum())

Pays                                                    0
Disponibilité alimentaire                               0
Exportations                                            0
Importations                                            0
Consommation                                            0
Production                                              0
Nombre d'habitants                                      0
Evolution de la population (en %) entre 2000 et 2018    2
dtype: int64


In [33]:
# Identification des valeurs manquantes
Valeurs_dispo_population=df_dispo_population[df_dispo_population.isnull().any(axis=1)]
Valeurs_dispo_population

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018
100,Monténégro,15.98,0.00,8.00,10.00,4.00,627563.00,NaN
138,Serbie,10.16,7.00,12.00,90.00,85.00,8829628.00,NaN


## <span style='color:green'> III - Fichier PIB</span>

In [34]:
#Import du fichier
# PIB = Somme des valeurs ajoutées + TVA + Droits et taxes sur les importations – Subventions sur les produits
# Le PIB par habitant d'un pays se calcule en divisant le PIB du pays par sa population totale.
# Cet indicateur utilise le PIB selon les prix courants. le PIB par habitant mesure le niveau de vie
PIB=pd.read_excel('PIB.xls')
PIB.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,4,Afghanistan,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,I$,2058.40,X,Ciffre de sources internationales,NaN
1,FS,Données de la sécurité alimentaire,710,Afrique du Sud,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,I$,13860.30,X,Ciffre de sources internationales,NaN
2,FS,Données de la sécurité alimentaire,8,Albanie,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,I$,12771.00,X,Ciffre de sources internationales,NaN
3,FS,Données de la sécurité alimentaire,12,Algérie,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,I$,11737.40,X,Ciffre de sources internationales,NaN
4,FS,Données de la sécurité alimentaire,276,Allemagne,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,I$,53071.50,X,Ciffre de sources internationales,NaN


In [35]:
# Vérification des informations du data frame
PIB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            187 non-null    object 
 1   Domaine                 187 non-null    object 
 2   Code zone (M49)         187 non-null    int64  
 3   Zone                    187 non-null    object 
 4   Code Élément            187 non-null    int64  
 5   Élément                 187 non-null    object 
 6   Code Produit            187 non-null    int64  
 7   Produit                 187 non-null    object 
 8   Code année              187 non-null    int64  
 9   Année                   187 non-null    int64  
 10  Unité                   187 non-null    object 
 11  Valeur                  187 non-null    float64
 12  Symbole                 187 non-null    object 
 13  Description du Symbole  187 non-null    object 
 14  Note                    0 non-null      fl

In [36]:
# Vérification des statistiques des colonnes
PIB.describe()

,Code zone (M49),Code Élément,Code Produit,Code année,Année,Valeur,Note
count,187.00,187.00,187.00,187.00,187.00,187.00,0.00
mean,426.14,6126.00,22013.00,2017.00,2017.00,20925.34,NaN
std,250.96,0.00,0.00,0.00,0.00,22086.26,NaN
min,4.00,6126.00,22013.00,2017.00,2017.00,773.60,NaN
25%,210.00,6126.00,22013.00,2017.00,2017.00,4862.70,NaN
50%,426.00,6126.00,22013.00,2017.00,2017.00,13102.10,NaN
75%,638.00,6126.00,22013.00,2017.00,2017.00,29087.35,NaN
max,894.00,6126.00,22013.00,2017.00,2017.00,126144.00,NaN


In [37]:
# Sélection des colonnes à conserver
PIB=PIB[['Zone','Valeur']]
PIB= PIB.rename(columns={'Zone':'Pays','Valeur':'PIB'})
PIB.head()

,Pays,PIB
0,Afghanistan,2058.40
1,Afrique du Sud,13860.30
2,Albanie,12771.00
3,Algérie,11737.40
4,Allemagne,53071.50


In [38]:
# Création d'une liste des pays
Liste_pays_PIB=PIB['Pays'].unique()
Liste_pays_PIB

array(['Afghanistan', 'Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne',
       'Angola', 'Antigua-et-Barbuda', 'Arabie saoudite', 'Argentine',
       'Arménie', 'Australie', 'Autriche', 'Azerbaïdjan', 'Bahamas',
       'Bahreïn', 'Bangladesh', 'Barbade', 'Bélarus', 'Belgique',
       'Belize', 'Bénin', 'Bermudes', 'Bhoutan',
       'Bolivie (État plurinational de)', 'Bosnie-Herzégovine',
       'Botswana', 'Brésil', 'Brunéi Darussalam', 'Bulgarie',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodge', 'Cameroun',
       'Canada', 'Chili', 'Chine', 'Chine - RAS de Hong-Kong',
       'Chine - RAS de Macao', 'Chypre', 'Colombie', 'Comores', 'Congo',
       'Costa Rica', "Côte d'Ivoire", 'Croatie', 'Danemark', 'Djibouti',
       'Dominique', 'Égypte', 'El Salvador', 'Émirats arabes unis',
       'Équateur', 'Espagne', 'Estonie', 'Eswatini',
       "États-Unis d'Amérique", 'Éthiopie', 'Fédération de Russie',
       'Fidji', 'Finlande', 'France', 'Gabon', 'Gambie', 'Géorgie',
      

In [39]:
# Renommer Chine par Chine, continentale
PIB['Pays']= PIB['Pays'].replace(["Chine"],["Chine, continentale"])

In [40]:
# Comparaison des 2 listes de pays
Différences_pays_dispo_PIB = PIB.loc[~PIB['Pays'].isin(Liste_pays_dispo)]
Différences_pays_dispo_PIB['Pays'].unique()

array(['Bahreïn', 'Bermudes', 'Bhoutan', 'Brunéi Darussalam', 'Burundi',
       'Comores', 'Guinée équatoriale', 'Îles Marshall', 'Libye',
       'Micronésie (États fédérés de)', 'Nauru', 'Palaos', 'Palestine',
       'Papouasie-Nouvelle-Guinée', 'Porto Rico', 'Qatar',
       'République démocratique du Congo', 'Seychelles', 'Singapour',
       'Somalie', 'Tonga', 'Tuvalu'], dtype=object)

In [41]:
# Nombre de pays différents
Différences_pays_dispo_PIB['Pays'].nunique()

22

In [42]:
# Détection des valeurs manquantes
print(PIB.isnull().sum())

Pays    0
PIB     0
dtype: int64


Il n'y a pas de valeurs manquantes.

In [43]:
# Détection des doublons
PIB.loc[PIB['Pays'].duplicated(keep=False),:]

,Pays,PIB


Il n'y a pas de doublons.

### _- Fusion des fichiers df_dispo_population & PIB_

In [44]:
# Jointure à gauche, 'left' utilise toutes les combinaisons de clés figurant dans le tableau de gauche
df_dispo_population_PIB=pd.merge(df_dispo_population,PIB,on=['Pays'],how='left')
df_dispo_population_PIB.head()

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB
0,Afghanistan,1.53,132.19,29.00,55.00,28.00,36296113.00,78.88,2058.40
1,Afrique du Sud,35.69,63.00,514.00,2035.00,1667.00,57009756.00,28.52,13860.30
2,Albanie,16.36,0.00,38.00,47.00,13.00,2884169.00,-7.88,12771.00
3,Algérie,6.38,0.00,2.00,264.00,275.00,41389189.00,36.04,11737.40
4,Allemagne,19.47,646.00,842.00,1609.00,1514.00,82658409.00,2.12,53071.50


## <span style='color:green'> IV - Fichier Stabilité politique</span>

In [45]:
# Import du fichier
Stabilité_politique=pd.read_excel('Stabilité_politique.xls')
Stabilité_politique.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,4,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-2.80,X,Ciffre de sources internationales,NaN
1,FS,Données de la sécurité alimentaire,710,Afrique du Sud,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-0.28,X,Ciffre de sources internationales,NaN
2,FS,Données de la sécurité alimentaire,8,Albanie,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,0.38,X,Ciffre de sources internationales,NaN
3,FS,Données de la sécurité alimentaire,12,Algérie,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-0.92,X,Ciffre de sources internationales,NaN
4,FS,Données de la sécurité alimentaire,276,Allemagne,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,0.59,X,Ciffre de sources internationales,NaN


In [46]:
# Vérification des informations du data frame
Stabilité_politique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            197 non-null    object 
 1   Domaine                 197 non-null    object 
 2   Code zone (M49)         197 non-null    int64  
 3   Zone                    197 non-null    object 
 4   Code Élément            197 non-null    int64  
 5   Élément                 197 non-null    object 
 6   Code Produit            197 non-null    int64  
 7   Produit                 197 non-null    object 
 8   Code année              197 non-null    int64  
 9   Année                   197 non-null    int64  
 10  Unité                   197 non-null    object 
 11  Valeur                  197 non-null    float64
 12  Symbole                 197 non-null    object 
 13  Description du Symbole  197 non-null    object 
 14  Note                    0 non-null      fl

In [47]:
# Vérification des statistiques des colonnes
Stabilité_politique.describe()

,Code zone (M49),Code Élément,Code Produit,Code année,Année,Valeur,Note
count,197.00,197.00,197.00,197.00,197.00,197.00,0.00
mean,426.88,6125.00,21032.00,2017.00,2017.00,-0.06,NaN
std,255.16,0.00,0.00,0.00,0.00,0.99,NaN
min,4.00,6125.00,21032.00,2017.00,2017.00,-2.94,NaN
25%,208.00,6125.00,21032.00,2017.00,2017.00,-0.65,NaN
50%,422.00,6125.00,21032.00,2017.00,2017.00,0.04,NaN
75%,643.00,6125.00,21032.00,2017.00,2017.00,0.77,NaN
max,894.00,6125.00,21032.00,2017.00,2017.00,1.92,NaN


In [48]:
# Sélection des colonnes à conserver
Stabilité_politique=Stabilité_politique[['Zone','Valeur']]
Stabilité_politique= Stabilité_politique.rename(columns={'Zone':'Pays','Valeur':"Stabilité politique"})
Stabilité_politique.head()

,Pays,Stabilité politique
0,Afghanistan,-2.80
1,Afrique du Sud,-0.28
2,Albanie,0.38
3,Algérie,-0.92
4,Allemagne,0.59


In [49]:
# Création d'une liste des pays
Liste_pays_Stabilité_politique=Stabilité_politique['Pays'].unique()
Liste_pays_Stabilité_politique

array(['Afghanistan', 'Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne',
       'Andorre', 'Angola', 'Antigua-et-Barbuda', 'Arabie saoudite',
       'Argentine', 'Arménie', 'Australie', 'Autriche', 'Azerbaïdjan',
       'Bahamas', 'Bahreïn', 'Bangladesh', 'Barbade', 'Bélarus',
       'Belgique', 'Belize', 'Bénin', 'Bermudes', 'Bhoutan',
       'Bolivie (État plurinational de)', 'Bosnie-Herzégovine',
       'Botswana', 'Brésil', 'Brunéi Darussalam', 'Bulgarie',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodge', 'Cameroun',
       'Canada', 'Chili', 'Chine - RAS de Hong-Kong',
       'Chine - RAS de Macao', 'Chine, Taiwan Province de', 'Chypre',
       'Colombie', 'Comores', 'Congo', 'Costa Rica', "Côte d'Ivoire",
       'Croatie', 'Cuba', 'Danemark', 'Djibouti', 'Dominique', 'Égypte',
       'El Salvador', 'Émirats arabes unis', 'Équateur', 'Érythrée',
       'Espagne', 'Estonie', 'Eswatini', "États-Unis d'Amérique",
       'Éthiopie', 'Fédération de Russie', 'Fidji', 'Finlan

In [50]:
# Comparaison des 2 listes de pays
Différence_pays = Stabilité_politique.loc[~Stabilité_politique['Pays'].isin(Liste_pays_dispo)]
Différence_pays['Pays'].unique()

array(['Andorre', 'Bahreïn', 'Bermudes', 'Bhoutan', 'Brunéi Darussalam',
       'Burundi', 'Comores', 'Érythrée', 'Groenland',
       'Guinée équatoriale', 'Îles Marshall', 'Libye',
       'Micronésie (États fédérés de)', 'Nauru', 'Palaos', 'Palestine',
       'Papouasie-Nouvelle-Guinée', 'Porto Rico', 'Qatar',
       'République arabe syrienne', 'République démocratique du Congo',
       'Samoa américaines', 'Seychelles', 'Singapour', 'Somalie',
       'Soudan du Sud', 'Tonga', 'Tuvalu'], dtype=object)

In [51]:
# Détection des valeurs manquantes
print(Stabilité_politique.isnull().sum())

Pays                   0
Stabilité politique    0
dtype: int64


Il n'y a pas de valeurs manquantes.

In [52]:
# Détection des doublons
Stabilité_politique.loc[Stabilité_politique['Pays'].duplicated(keep=False),:]

,Pays,Stabilité politique


Il n'y a pas de doublons.

### _- Fusion des fichiers df_dispo_population_PIB & Stabilité_politique_

In [53]:
# Jointure à gauche, 'left' utilise toutes les combinaisons de clés figurant dans le tableau de gauche
df_final=pd.merge(df_dispo_population_PIB,Stabilité_politique,on=['Pays'],how='left')
df_final.head()

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB,Stabilité politique
0,Afghanistan,1.53,132.19,29.00,55.00,28.00,36296113.00,78.88,2058.40,-2.80
1,Afrique du Sud,35.69,63.00,514.00,2035.00,1667.00,57009756.00,28.52,13860.30,-0.28
2,Albanie,16.36,0.00,38.00,47.00,13.00,2884169.00,-7.88,12771.00,0.38
3,Algérie,6.38,0.00,2.00,264.00,275.00,41389189.00,36.04,11737.40,-0.92
4,Allemagne,19.47,646.00,842.00,1609.00,1514.00,82658409.00,2.12,53071.50,0.59


In [54]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 10 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Pays                                                  172 non-null    object 
 1   Disponibilité alimentaire                             172 non-null    float64
 2   Exportations                                          172 non-null    float64
 3   Importations                                          172 non-null    float64
 4   Consommation                                          172 non-null    float64
 5   Production                                            172 non-null    float64
 6   Nombre d'habitants                                    172 non-null    float64
 7   Evolution de la population (en %) entre 2000 et 2018  170 non-null    float64
 8   PIB                                                   165 no

In [55]:
# Détection des valeurs manquantes
print(df_final.isnull().sum())

Pays                                                    0
Disponibilité alimentaire                               0
Exportations                                            0
Importations                                            0
Consommation                                            0
Production                                              0
Nombre d'habitants                                      0
Evolution de la population (en %) entre 2000 et 2018    2
PIB                                                     7
Stabilité politique                                     3
dtype: int64


In [56]:
# Identification des valeurs manquantes
Valeurs_manquantes_df_final=df_final[df_final.isnull().any(axis=1)]
Valeurs_manquantes_df_final

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB,Stabilité politique
33,"Chine, Taiwan Province de",33.17,8.00,161.00,785.00,652.00,23674546.00,8.01,NaN,0.86
34,"Chine, continentale",12.33,576.00,452.00,17518.00,18236.00,1421021791.00,10.62,14520.80,NaN
40,Cuba,23.72,132.19,312.00,269.00,29.00,11339254.00,1.90,NaN,0.65
100,Monténégro,15.98,0.00,8.00,10.00,4.00,627563.00,NaN,19682.30,-0.06
108,Nouvelle-Calédonie,38.71,132.19,9.00,11.00,1.00,277150.00,28.87,NaN,NaN
120,Polynésie française,47.40,0.00,15.00,13.00,1.00,276102.00,15.37,NaN,NaN
131,République populaire démocratique de Corée,1.62,0.00,0.00,41.00,43.00,25429825.00,11.43,NaN,-0.50
138,Serbie,10.16,7.00,12.00,90.00,85.00,8829628.00,NaN,16611.00,0.09
161,Venezuela (République bolivarienne du),20.28,0.00,25.00,596.00,600.00,29402484.00,19.41,NaN,-1.27
163,Yémen,8.53,0.00,78.00,237.00,168.00,27834819.00,63.70,NaN,-2.94


In [57]:
# Remplacement de valeurs nulles
df_final.loc[33, 'PIB'] = 24408
df_final.loc[34, 'PIB'] = 14520.8
df_final.loc[40, 'PIB'] = 12300
df_final.loc[108, 'PIB'] = 31100 
df_final.loc[120, 'PIB'] = 17000 
df_final.loc[131, 'PIB'] = 40957.4
df_final.loc[161, 'PIB'] = 17605
df_final.loc[163, 'PIB'] = 2507 


In [58]:
# Identification des valeurs manquantes
Valeurs_manquantes_df_final=df_final[df_final.isnull().any(axis=1)]
Valeurs_manquantes_df_final

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB,Stabilité politique
34,"Chine, continentale",12.33,576.00,452.00,17518.00,18236.00,1421021791.00,10.62,14520.80,NaN
100,Monténégro,15.98,0.00,8.00,10.00,4.00,627563.00,NaN,19682.30,-0.06
108,Nouvelle-Calédonie,38.71,132.19,9.00,11.00,1.00,277150.00,28.87,31100.00,NaN
120,Polynésie française,47.40,0.00,15.00,13.00,1.00,276102.00,15.37,17000.00,NaN
138,Serbie,10.16,7.00,12.00,90.00,85.00,8829628.00,NaN,16611.00,0.09


In [59]:
# Remplacement des autres valeurs par la moyenne de leurs colonnes
df_final=df_final.fillna(df_final.mean())
df_final

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB,Stabilité politique
0,Afghanistan,1.53,132.19,29.00,55.00,28.00,36296113.00,78.88,2058.40,-2.80
1,Afrique du Sud,35.69,63.00,514.00,2035.00,1667.00,57009756.00,28.52,13860.30,-0.28
2,Albanie,16.36,0.00,38.00,47.00,13.00,2884169.00,-7.88,12771.00,0.38
3,Algérie,6.38,0.00,2.00,264.00,275.00,41389189.00,36.04,11737.40,-0.92
4,Allemagne,19.47,646.00,842.00,1609.00,1514.00,82658409.00,2.12,53071.50,0.59
...,...,...,...,...,...,...,...,...,...,...
167,Émirats arabes unis,43.47,94.00,433.00,412.00,48.00,9487203.00,207.30,67183.60,0.62
168,Équateur,19.31,0.00,0.00,324.00,340.00,16785361.00,34.72,11617.90,-0.07
169,États-Unis d'Amérique,55.68,3692.00,123.00,18100.00,21914.00,325084756.00,16.11,59914.80,0.29
170,Éthiopie,0.13,132.19,1.00,14.00,14.00,106399924.00,64.93,2021.60,-1.68


In [60]:
# Identification de la France

df_final_france=df_final.loc[df_final['Pays']== 'France']
df_final_france

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB,Stabilité politique
51,France,22.90,501.00,506.00,1485.00,1750.00,64842509.00,10.13,44577.10,0.28


In [61]:
# Suppression de la France, notre entreprise étant basée dans ce pays et cherchant à s'exporter.
df_final=df_final.drop(index=([51]))

In [62]:
# Vérifications des infos du data frame final
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 171
Data columns (total 10 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Pays                                                  171 non-null    object 
 1   Disponibilité alimentaire                             171 non-null    float64
 2   Exportations                                          171 non-null    float64
 3   Importations                                          171 non-null    float64
 4   Consommation                                          171 non-null    float64
 5   Production                                            171 non-null    float64
 6   Nombre d'habitants                                    171 non-null    float64
 7   Evolution de la population (en %) entre 2000 et 2018  171 non-null    float64
 8   PIB                                                   171 no

In [63]:
# Vérifications des statistiques des colonnes
df_final.describe()

,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB,Stabilité politique
count,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00,171.00
mean,20.20,130.03,87.09,652.21,719.20,42713095.26,31.30,20187.47,-0.07
std,15.91,455.27,183.34,2129.30,2478.03,153503911.61,30.20,20428.76,0.90
min,0.13,0.00,0.00,2.00,0.00,52045.00,-20.01,912.80,-2.94
25%,6.42,0.00,3.00,29.00,14.50,2864791.50,10.24,5068.90,-0.62
50%,17.98,8.00,16.00,101.00,72.00,9729823.00,26.41,13589.70,-0.04
75%,30.12,132.19,83.00,376.00,478.00,29609625.00,48.72,28639.25,0.65
max,72.31,4223.00,1069.00,18100.00,21914.00,1421021791.00,207.30,126144.00,1.60


In [64]:
#L’IQR ou écart interquartile en Français est une mesure de dispersion des données.
# Cela correspond à la différence entre le 3ème et le 1er quartile (Q3 et Q1, les limites de la “boîte” à moustache).
# Le premier quartile est la plus petite valeur Q_{1} de la série de données tel que 25% au moins des autres valeurs
#lui sont inférieures ou égales.
# Le troisième quartile est la plus petite valeur Q_{3} de la série de données tel que 75% au moins des autres valeurs lui 
# sont inférieures ou égales.
# On considère qu’une valeur est extrême (mais pas forcément aberrante) si sa valeur est inférieure à la valeur du Q1-1,5 fois
# l’IQR, ou supérieure au Q3 + 1,5 fois l’IQR.  
# Ce sont des valeurs qui sont très éloignées de la majorité des autres valeurs, ce qui les rend “suspectes”.


Q1 = df_final["Nombre d'habitants"].quantile(q=0.25)
Q3 = df_final["Nombre d'habitants"].quantile(q=0.75)
Iq = Q3 - Q1

limite_outliers_inf = Q1-1.5*Iq
limite_outliers_sup = Q3+1.5*Iq

outliers_inf = df_final["Nombre d'habitants"] <limite_outliers_inf
outliers_sup = df_final["Nombre d'habitants"] > limite_outliers_sup

print("Limite pour les outliers inférieurs:\n", round(limite_outliers_inf,2))
print("Limite pour les outliers supérieurs:\n", round(limite_outliers_sup,2))

Limite pour les outliers inférieurs:
 -37252458.75
Limite pour les outliers supérieurs:
 69726875.25


In [65]:
print ("La limite inférieure est négative, elle est de", round(limite_outliers_inf,2), "€, il n'y a pas d'outliers inférieurs")
print("La limite supérieure est de", round(limite_outliers_sup,2), "€. Le nombre d'habitants maximum est de 1 421 021 791, on sait donc qu'il y a au moins un outlier supérieur, on les identifie dans un nouveau data frame.")


La limite inférieure est négative, elle est de -37252458.75 €, il n'y a pas d'outliers inférieurs
La limite supérieure est de 69726875.25 €. Le nombre d'habitants maximum est de 1 421 021 791, on sait donc qu'il y a au moins un outlier supérieur, on les identifie dans un nouveau data frame.


In [66]:
# Utilisation fonction .loc pour identifier les lignes
# Utilisation de la méthode sort_values pour trier la colonne nombre d'habitants en fonction des valeurs, dans l'ordre décroissant
df_final_outliers=df_final.loc[(df_final["Nombre d'habitants"]>limite_outliers_sup)] 
df_final_outliers.sort_values(by=["Nombre d'habitants"], ascending = False).head()

,Pays,Disponibilité alimentaire,Exportations,Importations,Consommation,Production,Nombre d'habitants,Evolution de la population (en %) entre 2000 et 2018,PIB,Stabilité politique
34,"Chine, continentale",12.33,576.00,452.00,17518.00,18236.00,1421021791.00,10.62,14520.80,-0.06
66,Inde,2.22,4.00,0.00,2965.00,3545.00,1338676785.00,28.02,6182.90,-0.76
169,États-Unis d'Amérique,55.68,3692.00,123.00,18100.00,21914.00,325084756.00,16.11,59914.80,0.29
67,Indonésie,7.19,0.00,1.00,1904.00,2301.00,264650963.00,26.55,10935.60,-0.50
114,Pakistan,5.86,4.00,2.00,1218.00,1281.00,207906209.00,49.10,5091.10,-2.41


On constate que 2 pays, la Chine et l'Inde ont une population au-delà d'1 milliard d'habitants, nous les garderons pour l'instant dans notre analyse.

In [67]:
# Enregistrement du fichier final
df_final.to_csv("C:/Users/charl/df_final.csv", index=False)  